# Feature Engineering - Users (Versão Spark)

**Conversão do notebook pandas → Spark**

Este notebook:
1. Lê users da camada Bronze (Spark)
2. Filtra apenas usuários que têm reviews
3. Processa features (num_friends, compliments, etc)
4. Aplica normalização (log + MinMaxScaler)
5. Salva features finais na Silver layer

**Vantagens vs Pandas:**
- ✅ Sem problemas de memória
- ✅ Processamento paralelo
- ✅ Acessa datalake diretamente
- ✅ Escalável

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, count, split, size, log1p, 
    when, coalesce, lit, broadcast,
    datediff, current_date, to_timestamp,
    min as spark_min, max as spark_max
)
from pyspark.sql.types import DoubleType, IntegerType
import pyspark.sql.functions as F

In [3]:
# ⚡ CONFIGURAÇÃO SPARK OTIMIZADA
spark = SparkSession.builder \
    .appName("User Feature Engineering") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.shuffle.partitions", "200") \
    .getOrCreate()

print(f"✅ Spark version: {spark.version}")

✅ Spark version: 3.5.0


In [4]:
# Configuração de Paths
BASE_PATH = '/home/jovyan/work'
DATA_PATH = f'{BASE_PATH}/data'
BRONZE_PATH = f'{DATA_PATH}/bronze'
SILVER_PATH = f'{DATA_PATH}/silver'
GOLD_PATH = f'{DATA_PATH}/gold'

print(f"🥉 Bronze: {BRONZE_PATH}")
print(f"🥈 Silver: {SILVER_PATH}")
print(f"🥇 Gold: {GOLD_PATH}")

🥉 Bronze: /home/jovyan/work/data/bronze
🥈 Silver: /home/jovyan/work/data/silver
🥇 Gold: /home/jovyan/work/data/gold


## 1. Carregar Dados do Datalake

In [5]:
print("📥 [1/5] Carregando dados do datalake...\n")

# 1. Users (Bronze)
print("   📦 Carregando users...")
df_users = spark.read.parquet(f"{BRONZE_PATH}/user")
print(f"      ✅ Users: {df_users.count():,} registros")

# 2. Reviews (para filtrar usuários relevantes)
print("   📦 Carregando reviews para filtrar usuários...")
df_reviews = spark.read.parquet(f"{SILVER_PATH}/review") \
    .select("user_id") \
    .distinct()
print(f"      ✅ Usuários únicos em reviews: {df_reviews.count():,}")

print("\n✅ Dados carregados!")

📥 [1/5] Carregando dados do datalake...

   📦 Carregando users...
      ✅ Users: 1,987,897 registros
   📦 Carregando reviews para filtrar usuários...
      ✅ Usuários únicos em reviews: 56,555

✅ Dados carregados!


In [6]:
# Verificar schema dos users
print("📋 Schema do Users:\n")


print("\n📊 Amostra:")
df_users.show(1, truncate=30, vertical=True)

📋 Schema do Users:


📊 Amostra:
-RECORD 0--------------------------------------------
 average_stars      | 4.73                           
 compliment_cool    | 1                              
 compliment_cute    | 0                              
 compliment_funny   | 1                              
 compliment_hot     | 0                              
 compliment_list    | 0                              
 compliment_more    | 0                              
 compliment_note    | 0                              
 compliment_photos  | 0                              
 compliment_plain   | 0                              
 compliment_profile | 0                              
 compliment_writer  | 0                              
 cool               | 1                              
 elite              |                                
 fans               | 1                              
 friends            | 5-r8vdBoqMoHdHsAMtXw3Q, 816... 
 funny              | 3                           

## 2. Filtrar Usuários Relevantes

Manter apenas usuários que possuem reviews no dataset filtrado.

In [7]:
print("\n🔧 [2/5] Filtrando usuários relevantes...\n")

# Colunas que queremos manter
cols_to_keep = [
    'user_id',
    'review_count',
    'yelping_since',
    'useful',
    'funny',
    'cool',
    'fans',
    'average_stars',
    'compliment_hot',
    'compliment_more',
    'compliment_profile',
    'friends'
]

# Filtrar apenas usuários com reviews (broadcast join para otimização)
df_users_filtered = df_users \
    .join(broadcast(df_reviews), on="user_id", how="inner") \
    .select(cols_to_keep)

print(f"   ✅ Usuários após filtro: {df_users_filtered.count():,}")


🔧 [2/5] Filtrando usuários relevantes...

   ✅ Usuários após filtro: 56,555


## 3. Feature Engineering

- Contar número de amigos (campo `friends` é string separada por vírgula)
- Converter `yelping_since` para timestamp
- Calcular idade da conta
- Agregar compliments

In [8]:
print("\n🔧 [3/5] Feature Engineering...\n")

# 1. Contar número de amigos
# O campo 'friends' é uma string separada por vírgulas ou 'None'
print("   ⚙️ Calculando num_friends...")
df_features = df_users_filtered.withColumn(
    "num_friends",
    when(
        (col("friends").isNull()) | (col("friends") == "None") | (col("friends") == ""),
        0
    ).otherwise(
        size(split(col("friends"), ","))
    )
).drop("friends")  # Remover string gigante

# 2. Converter yelping_since para timestamp
print("   ⚙️ Convertendo yelping_since...")
df_features = df_features.withColumn(
    "yelping_since",
    to_timestamp(col("yelping_since"))
)

# 3. Calcular idade da conta em dias
print("   ⚙️ Calculando account_age_days...")
df_features = df_features.withColumn(
    "account_age_days",
    datediff(current_date(), col("yelping_since"))
)

# 4. Agregar compliments
print("   ⚙️ Agregando compliments...")
df_features = df_features.withColumn(
    "total_compliments",
    coalesce(col("compliment_hot"), lit(0)) + 
    coalesce(col("compliment_more"), lit(0)) + 
    coalesce(col("compliment_profile"), lit(0))
)

print("\n   ✅ Features criadas!")

# Mostrar amostra
df_features.select(
    "user_id", "review_count", "useful", "fans", 
    "num_friends", "total_compliments", "account_age_days"
).show(5)


🔧 [3/5] Feature Engineering...

   ⚙️ Calculando num_friends...
   ⚙️ Convertendo yelping_since...
   ⚙️ Calculando account_age_days...
   ⚙️ Agregando compliments...

   ✅ Features criadas!
+--------------------+------------+------+----+-----------+-----------------+----------------+
|             user_id|review_count|useful|fans|num_friends|total_compliments|account_age_days|
+--------------------+------------+------+----+-----------+-----------------+----------------+
|vvSaOV1MVVX6zX4T0...|         386|  1156|  21|        265|               12|            4659|
|o4iHwVDRfhMMlHF6I...|          84|    91|   7|         95|                4|            3942|
|hME-y8rH5MUc-vpN2...|          89|   191|  19|        139|                4|            2553|
|0tJbuiMAkcbzd6Lkb...|          31|    21|   0|          8|                0|            5061|
|QkXdLvoYSm8bS9XqB...|          45|    68|   0|          5|                1|            6863|
+--------------------+------------+------+----+-

## 4. Normalização

- Log transformation para features com distribuição assimétrica
- Min-Max scaling para normalizar entre 0 e 1

In [9]:
print("\n🔧 [4/5] Aplicando transformações...\n")

# 1. Log transformation (log1p para evitar log(0))
print("   ⚙️ Aplicando log1p...")
log_columns = ['review_count', 'useful', 'fans', 'num_friends', 'total_compliments']

for col_name in log_columns:
    df_features = df_features.withColumn(
        f"{col_name}_log",
        log1p(col(col_name))
    )

print("   ✅ Log transformation aplicada!")


🔧 [4/5] Aplicando transformações...

   ⚙️ Aplicando log1p...
   ✅ Log transformation aplicada!


In [10]:
# 2. Min-Max Scaling
print("   ⚙️ Aplicando Min-Max scaling...")

cols_to_normalize = [
    'review_count_log', 'useful_log', 'fans_log', 
    'num_friends_log', 'total_compliments_log', 'account_age_days'
]

# Calcular min/max e normalizar
for col_name in cols_to_normalize:
    stats = df_features.agg(
        spark_min(col_name).alias("min_val"),
        spark_max(col_name).alias("max_val")
    ).collect()[0]
    
    min_val = stats["min_val"] or 0
    max_val = stats["max_val"] or 1
    
    if max_val != min_val:
        df_features = df_features.withColumn(
            col_name,
            (col(col_name) - lit(min_val)) / lit(max_val - min_val)
        )
    else:
        df_features = df_features.withColumn(col_name, lit(0.0))

# Normalizar average_stars (1-5 → 0-1)
df_features = df_features.withColumn(
    "average_stars",
    col("average_stars") / 5.0
)

print("   ✅ Min-Max scaling aplicado!")

   ⚙️ Aplicando Min-Max scaling...
   ✅ Min-Max scaling aplicado!


In [11]:
# Selecionar colunas finais
final_columns = [
    'user_id',
    'review_count_log',
    'useful_log',
    'fans_log',
    'num_friends_log',
    'total_compliments_log',
    'average_stars',
    'account_age_days'
]

df_final = df_features.select(final_columns)

# Cache para reutilização
df_final.cache()

print(f"\n   ✅ Dataset final: {df_final.count():,} usuários")
print(f"   📋 Colunas: {final_columns}")


   ✅ Dataset final: 56,555 usuários
   📋 Colunas: ['user_id', 'review_count_log', 'useful_log', 'fans_log', 'num_friends_log', 'total_compliments_log', 'average_stars', 'account_age_days']


In [12]:
# Preview do resultado
print("\n📊 Preview das features finais:")
df_final.show(2, truncate=False, vertical = True)


📊 Preview das features finais:
-RECORD 0---------------------------------------
 user_id               | vvSaOV1MVVX6zX4T0NDl_Q 
 review_count_log      | 0.5801752730612584     
 useful_log            | 0.5764112229365866     
 fans_log              | 0.3276726721873028     
 num_friends_log       | 0.5806743062219525     
 total_compliments_log | 0.24733237608934172    
 average_stars         | 0.8140000000000001     
 account_age_days      | 0.5131871623768669     
-RECORD 1---------------------------------------
 user_id               | o4iHwVDRfhMMlHF6IlzrJQ 
 review_count_log      | 0.4131538263548776     
 useful_log            | 0.3695155605054942     
 fans_log              | 0.22043571930861822    
 num_friends_log       | 0.4746846013433325     
 total_compliments_log | 0.1551945272886602     
 average_stars         | 0.892                  
 account_age_days      | 0.3992691452176676     
only showing top 2 rows



## 5. Salvar na Silver Layer

In [19]:
print("\n💾 [5/5] Salvando na Silver Layer...\n")

import os
import shutil

# Criar diretório Silver se não existir
os.makedirs(SILVER_PATH, exist_ok=True)

output_path = f'{SILVER_PATH}/user_features'

# Remover se existir
if os.path.exists(output_path):
    shutil.rmtree(output_path)
    print(f"   🗑️  Removido arquivo antigo: {output_path}")

# Salvar
df_final \
    .repartition(10) \
    .write \
    .mode('overwrite') \
    .option('compression', 'snappy') \
    .parquet(output_path)

print(f"\n{'='*60}")
print(f"✅ ARQUIVO FINAL GERADO: {output_path}")
print(f"   📊 Dimensões: {df_final.count():,} linhas x {len(df_final.columns)} colunas")
print(f"   📦 Partições: 10")
print(f"   🗜️  Compressão: SNAPPY")
print(f"{'='*60}")


💾 [5/5] Salvando na Silver Layer...


✅ ARQUIVO FINAL GERADO: /home/jovyan/work/data/silver/user_features
   📊 Dimensões: 56,555 linhas x 8 colunas
   📦 Partições: 10
   🗜️  Compressão: SNAPPY


In [20]:
# Verificar arquivo salvo
print("\n🔍 Verificando arquivo salvo...\n")

df_verify = spark.read.parquet(output_path)

print(f"✅ Arquivo lido com sucesso!")
print(f"   Total de registros: {df_verify.count():,}")
print(f"\n📋 Colunas:")
for c in df_verify.columns:
    print(f"   - {c}")

print("\n📊 Amostra dos dados:")
df_verify.show(5, truncate=10)


🔍 Verificando arquivo salvo...

✅ Arquivo lido com sucesso!
   Total de registros: 56,555

📋 Colunas:
   - user_id
   - review_count_log
   - useful_log
   - fans_log
   - num_friends_log
   - total_compliments_log
   - average_stars
   - account_age_days

📊 Amostra dos dados:
+----------+----------------+----------+----------+---------------+---------------------+-------------+----------------+
|   user_id|review_count_log|useful_log|  fans_log|num_friends_log|total_compliments_log|average_stars|account_age_days|
+----------+----------------+----------+----------+---------------+---------------------+-------------+----------------+
|8lcv5rv...|      0.50294...|0.50850...|0.22043...|     0.49071...|           0.24733...|        0.654|      0.56927...|
|Lnv6mh-...|      0.40082...|0.33857...|0.23292...|     0.53593...|                  0.0|        0.876|      0.42866...|
|qskILQ3...|      0.56194...|0.58767...|0.33238...|     0.43728...|           0.39480...|        0.736|      0.60470

In [ ]:
# Limpeza
print("\n🧹 Limpando cache...")
spark.catalog.clearCache()
print("✅ Cache limpo!")

print("\n🎉 PROCESSAMENTO COMPLETO!")

---

## 📊 Resumo do Pipeline

**Input:**
- `bronze/users` - Usuários brutos
- `silver/reviews` - Reviews filtradas (para obter user_ids relevantes)

**Transformações:**
1. ✅ Filtro de usuários com reviews
2. ✅ Contagem de amigos (num_friends)
3. ✅ Cálculo de idade da conta (account_age_days)
4. ✅ Agregação de compliments
5. ✅ Log transformation (review_count, useful, fans, etc)
6. ✅ MinMaxScaler em features numéricas

**Output:**
- `silver/user_features` - Features prontas para modelo

**Features Finais:**
- `user_id` - ID único do usuário
- `review_count_log` - Reviews normalizadas [0-1]
- `useful_log` - Votos úteis normalizados [0-1]
- `fans_log` - Fãs normalizados [0-1]
- `num_friends_log` - Amigos normalizados [0-1]
- `total_compliments_log` - Compliments normalizados [0-1]
- `average_stars` - Rating médio normalizado [0-1]
- `account_age_days` - Idade da conta normalizada [0-1]

---